In [2]:
from torch.utils.data import Dataset
import os
import matplotlib.pyplot as plt
import matplotlib.image as Mapping
import numpy as np
from PIL  import Image
import cs231n.datasets.Load_faces as dst
from torch.utils.data import DataLoader
import torchvision.transforms as T
import torch.nn as nn
import torch
import matplotlib.pyplot as plt
from torch import optim
from torch.utils.checkpoint import checkpoint_sequential
from sklearn.preprocessing import MinMaxScaler
from torch.utils.tensorboard import SummaryWriter
import torchvision

This application is used to convert notebook files (*.ipynb) to various other
formats.


Options

-------



Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.


--debug

    set log level to logging.DEBUG (maximize logging output)

--generate-config

    generate default config file

-y

    Answer yes to any questions instead of prompting.

--execute

    Execute the notebook prior to export.

--allow-errors

    Continue notebook execution even if one of the cells throws an error and include the error message in the cell output (the default behaviour is to abort conversion). This flag is only relevant if '--execute' was specified, too.

--stdin

    read a single notebook file from stdin. Write the resulting notebook with default basename 'notebook.*'

--stdout

    Write notebook output to stdout instead of files.

--inplace

    Run nbconvert in 

[NbConvertApp] WARNING | pattern 'Load_faces.ipynb' matched no files


In [3]:
dir_path = "C:/Users/wtser/Desktop/learnData/data/faces/"
transform = T.Compose([T.ToTensor(),T.Normalize([0.6756,0.5782,0.5534],[0.2733,0.2703,0.2562])])
train_data = dst.load_Face(root =dir_path,number = 6400,transform=transform)
device = torch.device('cuda')
writer = SummaryWriter(log_dir="E:/GANPIC/five/")


In [4]:
def normData(loader_data):
    for x,y in loader_data:
        x = x/2.4720
        yield (x,y)
    

In [5]:
loader_train = DataLoader(train_data,batch_size=32,shuffle=True)
# loader_train = normData(loader_train)

In [6]:
class SimpleCustomBatch:
    def __init__(self, data):
        transposed_data = list(zip(*data))
        print(transposed_data[1])
        self.inp = torch.stack(transposed_data[0], 0)
        self.tgt = torch.stack(transposed_data[1], 0)

    def pin_memory(self):
        self.inp = self.inp.pin_memory()
        self.tgt = self.tgt.pin_memory()
        return self

def collate_wrapper(batch):
    return SimpleCustomBatch(batch)


In [7]:
a = torch.randn(2,3,4).to(device)
# print(a)
a = a.unsqueeze(3)
a_ = a.permute(3,1,2,0)
a,a_ = torch.broadcast_tensors(a,a_)
n = a-a_
_sum = torch.sum(n,dim=2)
print(_sum)
eraser = torch.eye(2,device=device).view(2,1,2)
print(eraser)
eraser,_sum = torch.broadcast_tensors(eraser,_sum)
print(eraser)
cb = _sum+eraser*1e6


tensor([[[ 0.0000,  0.5964],
         [ 0.0000, -3.1986],
         [ 0.0000, -2.4024]],

        [[-0.5964,  0.0000],
         [ 3.1986,  0.0000],
         [ 2.4024,  0.0000]]], device='cuda:0')
tensor([[[1., 0.]],

        [[0., 1.]]], device='cuda:0')
tensor([[[1., 0.],
         [1., 0.],
         [1., 0.]],

        [[0., 1.],
         [0., 1.],
         [0., 1.]]], device='cuda:0')


In [8]:
class MinibatchDiscriminator(nn.Module):
    def __init__(self,insize,numkernel,dimsize):
        super(MinibatchDiscriminator,self).__init__()
        self.numkernel = numkernel
        self.dimsize = dimsize
        self.insize = insize
        self.layer = nn.Linear(insize,numkernel*dimsize)
    def forward(self,x):
        feature = self.layer(x).view(-1,self.numkernel,self.dimsize)
        feature = feature.unsqueeze(3)
        feature_ = feature.permute(3,1,2,0)
        feature,feature_ = torch.broadcast_tensors(feature,feature_)
        norm = torch.sum(torch.abs(feature-feature_),dim=2)
        eraser = torch.eye(feature.shape[0],device=torch.device('cuda')).view(feature.shape[0],1,feature.shape[0])
        eraser,norm = torch.broadcast_tensors(eraser,norm)
        c_b = torch.exp(-(norm+eraser*1e6))
        o_b = torch.sum(c_b,dim=2)
        x = torch.cat((x,o_b),dim=1)
        return x

In [9]:
class Reshape(nn.Module):
    def __init__(self,_shape):
        super(Reshape,self).__init__()
        self.shape = _shape
    
    def forward(self,x):
        return x.view(-1,*self.shape)

In [10]:
class GAN(nn.Module):
    def __init__(self,indim,img_shape):
        super(GAN,self).__init__()
        self.img_shape = img_shape
        
        def block(indim,outdim,normalize=True):
            layers = [nn.Linear(indim,outdim)]
            if normalize:
                layers.append(nn.BatchNorm1d(outdim,0.8))
            layers.append(nn.LeakyReLU(0.2,inplace=False))
            return layers
        
        self.model = nn.Sequential(nn.Linear(100,512*6*6),
                                   nn.ReLU(inplace=True),
                                   nn.BatchNorm1d(512*6*6),
                                   Reshape((512,6,6)),
#             nn.ConvTranspose2d(100,512,kernel_size=6,stride=1,padding=0,bias=False),
#                                    nn.ReLU(inplace=True),
                                   nn.ConvTranspose2d(512,256,kernel_size=4,stride=2,padding=1,bias=False),
                                   nn.BatchNorm2d(256),
                                   nn.ReLU(inplace=True),
                                   nn.ConvTranspose2d(256,128,kernel_size=4,stride=2,padding=1,bias=False),
                                   nn.BatchNorm2d(128),
                                   nn.ReLU(inplace=True),
                                   nn.ConvTranspose2d(128,64,kernel_size=4,stride=2,padding=1,bias=False),
                                   nn.BatchNorm2d(64),
                                   nn.ReLU(inplace=True),
                                   nn.ConvTranspose2d(64,3,kernel_size=4,stride=2,padding=1,bias=False),
                                   nn.Tanh()
                                  )
    def forward(self,x):
        out = self.model(x)
        return out

In [11]:
class Discriminator(nn.Module):
    def __init__(self,img_shape):
        super(Discriminator,self).__init__()
        self.model = nn.Sequential(nn.Conv2d(3,64,kernel_size=4,stride=2,padding=1,bias=False),
                                  nn.LeakyReLU(0.2,inplace=True),
                                  nn.Conv2d(64,128,kernel_size=4,stride=2,padding=1,bias=False),
                                  nn.BatchNorm2d(128),
                                  nn.LeakyReLU(0.2,inplace=True),
                                  nn.Conv2d(128,256,kernel_size=4,stride=2,padding=1,bias=False),
                                  nn.BatchNorm2d(256),
                                  nn.LeakyReLU(0.2,inplace=True),
                                  nn.Conv2d(256,512,kernel_size=4,stride=2,padding=1,bias=False),
                                  nn.BatchNorm2d(512),
                                  nn.LeakyReLU(0.2,inplace=True),
                                  )
        self.mnibt_dismtor = MinibatchDiscriminator(insize=18432,numkernel=50,dimsize=10)  
        self.lastlayer = nn.Linear(18432+50,1)
        
    def forward(self,x,matching=False):
        x = self.model(x)
        x = x.view(x.shape[0],-1)
        feature = self.mnibt_dismtor(x)
        out = self.lastlayer(feature)
        if matching:
            return feature,out
        return out

In [12]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [13]:
def save_model_dict(model,optimizer,path):
    checkpoint = {'model':model.state_dict(),'optimizer':optimizer.state_dict()}
    torch.save(checkpoint,path)

In [14]:
def load_model_dict(model,path):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model'])
    model = model.cuda()
    optimizer = optim.Adam(model.parameters(),lr = 2e-4,betas=(0.5, 0.999))
    optimizer.load_state_dict(checkpoint['optimizer'])
    return model,optimizer

In [18]:
generator = GAN(100,(3,96,96))
generator.apply(weights_init)
generator,optimizer_G = load_model_dict(generator,"E:/GANModel/DCGAN3/generator/generator_epoch28")
# generator = generator.cuda()

In [22]:
discriminator = Discriminator([3,96,96])
discriminator.apply(weights_init)
discriminator ,optimizer_D= load_model_dict(discriminator,"E:/GANModel/DCGAN3/discriminator/discriminator_epoch28")
# discriminator = discriminator.cuda()

In [23]:
epochs = 1
k = 3
criterion = nn.BCELoss()
criterion_G = nn.MSELoss()
# Optimizers
# optimizer_G = optim.Adam(generator.parameters(),lr = 2e-4,betas=(0.5, 0.999))
# optimizer_D = optim.Adam(discriminator.parameters(), lr=2e-4,betas=(0.5, 0.999))
# optimizer_G = optim.Adam(generator.parameters(),lr = 2e-4)
# optimizer_D = optim.Adam(discriminator.parameters(), lr=2e-4)
generator_path = "E:/GANModel/DCGAN3/generator/"
discriminator_path = "E:/GANModel/DCGAN3/discriminator/"

In [ ]:
for i in range(29,301):
    for index,(x,y) in enumerate(loader_train):
        fake = torch.zeros(x.shape[0],1,requires_grad=False).to(device)
        real = torch.rand(x.shape[0],1,requires_grad=False).to(device)
        x = x /2.4720
        for j in range(1):
            optimizer_D.zero_grad()
            # unsupervised loss
			# D(x) = Z(x) / {Z(x) + 1}, where Z(x) = \sum_{k=1}^K exp(l_k(x))
			# softplus(x) := log(1 + exp(x))
			# logD(x) = logZ(x) - log(Z(x) + 1)
			# 		  = logZ(x) - log(exp(log(Z(x))) + 1)
			# 		  = logZ(x) - softplus(logZ(x))
			# 1 - D(x) = 1 / {Z(x) + 1}
			# log{1 - D(x)} = log1 - log(Z(x) + 1)
			# 				= -log(exp(log(Z(x))) + 1)
			# 				= -softplus(logZ(x)
            
            #train discriminator by real
            real_data = x.to(device)
            output = discriminator(real_data.detach(),False)
            #因为在这里并没有 k 类，只有真和假，也就是说 k=1,所以 sum 操作可以忽略，然后 log 和 exp 又相互抵消，
            #所以相当于 log Z(x) = l(x)
            log_sum_exp = output
            log_d = log_sum_exp - torch.nn.functional.softplus(log_sum_exp)
            gloss_d_real = -1.0*torch.sum(log_d)/32
            
            #train discriminator by fake
            noise = torch.randn(x.shape[0],100,requires_grad=False).to(device)
            fake_img = generator(noise)
            output = discriminator(fake_img.detach(),False)
            log_sum_exp = output
            gloss_d_fake = torch.sum(torch.nn.functional.softplus(log_sum_exp))/32
            loss_unsupervised = gloss_d_real + gloss_d_fake
            gloss_d = loss_unsupervised
            gloss_d.backward()
            optimizer_D.step()
        #train generator
        optimizer_G.zero_grad()
        feature_fake,output_fake = discriminator(fake_img,True)
        log_sum_exp = output_fake
        log_d = log_sum_exp - torch.nn.functional.softplus(log_sum_exp)
        gloss_g = -1.0*torch.sum(log_d)/32
        #feature matching
        feature_fake,output_fake = discriminator(fake_img,True)
        feature_real,output_real = discriminator(real_data,True)
        gloss_g += criterion_G(feature_fake,feature_real).to(device)
        gloss_g.backward()
        optimizer_G.step()
    writer.add_scalars('loss',{'gloss_g':gloss_g,'gloss_d':gloss_d,'gloss_d_real':gloss_d_real,
                              'gloss_d_fake':gloss_d_fake},global_step=i)
    
    if i%50 == 0:
            print('gloass_g:{},gloss_d:{},gloss_d_real{},gloss_d_fake{}'.format(gloss_g,gloss_d,gloss_d_real,gloss_d_fake))
            store_img = torchvision.utils.make_grid((fake_img+1)/2.0,padding=2)
            writer.add_image('generate_img',store_img,global_step=i)
            
            g_path = os.path.join(generator_path,"generator_epoch{}".format(i))
            d_path = os.path.join(discriminator_path,"discriminator_epoch{}".format(i))
            save_model_dict(generator,optimizer_G,g_path)
            save_model_dict(discriminator,optimizer_D,d_path)
#             a = fake_img[0].cpu()
#             img_numpy = a.detach().numpy()
#             img_numpy = img_numpy.transpose(1,2,0)
#             print(img_numpy[:2,:2])
#             plt.imshow(((img_numpy+1.0)/2.0))
#             plt.show()
            writer.close()

gloass_g:10.949036598205566,gloss_d:0.009675819426774979,gloss_d_real0.009482424706220627,gloss_d_fake0.00019339470600243658
gloass_g:11.986099243164062,gloss_d:0.0019444769714027643,gloss_d_real0.0012950599193572998,gloss_d_fake0.0006494169938378036
gloass_g:11.670805931091309,gloss_d:0.0003154685255140066,gloss_d_real0.00011515617370605469,gloss_d_fake0.00020031236635986716
gloass_g:13.544977188110352,gloss_d:0.0010208911262452602,gloss_d_real0.0009246170520782471,gloss_d_fake9.627412509871647e-05
gloass_g:10.166580200195312,gloss_d:0.007071954198181629,gloss_d_real0.003454849123954773,gloss_d_fake0.0036171050742268562
gloass_g:17.98133659362793,gloss_d:0.006963432766497135,gloss_d_real0.0007461756467819214,gloss_d_fake0.006217257119715214
gloass_g:3.0592994689941406,gloss_d:0.3188855051994324,gloss_d_real0.07798594236373901,gloss_d_fake0.24089954793453217
gloass_g:10.935698509216309,gloss_d:0.0017050157766789198,gloss_d_real0.00035053491592407227,gloss_d_fake0.0013544808607548475
gl